# Frequency input
Put together the letter and bigram frequency files based on the data that comes from Mathieu. Distributes the frequency of combined characters that are not in the to-be-mapped character set of the scenario and filters out characters that are irrelevant.

In [1]:
from read_input import *
import codecs

In [7]:




#scenario="scenario1"
#derive_probabilities_from_raw_values(letter_file, bigram_file, scenario)

#scenario="scenario2"
#derive_probabilities_from_raw_values(letter_file, bigram_file, scenario)

scenario="scenario3"
letter_file = "input/resources/Code_all_Avg_freqs_1.tsv"
bigram_file = "input/resources/Code_all_Avg_freqs_2.tsv"
derive_probabilities_from_raw_values(letter_file, bigram_file, scenario, "_code")

letter_file = "input/resources/Twitter_all_Avg_freqs_1.tsv"
bigram_file = "input/resources/Twitter_all_Avg_freqs_2.tsv"
derive_probabilities_from_raw_values(letter_file, bigram_file, scenario, "_twitter")

letter_file = "input/resources/Formals_all_Avg_freqs_1.tsv"
bigram_file = "input/resources/Formals_all_Avg_freqs_2.tsv"
derive_probabilities_from_raw_values(letter_file, bigram_file, scenario, "_formal")



[u'\xe6', u'\xc6', u'\u0153', u'\u0152', u'\xdf', u'\u1e9e', u'\xfe', u'\xde', u'\xf0', u'\xd0', u'\u014b', u'\u014a', u'\u0133', u'\u0132', u'\u01dd', u'\u018e', u'\u0292', u'\u01b7', u'&', u'\u03b82', u'\xae', u'\xa9', u'\u017f', u'\xba', u'\xaa', u'\xb7', u'\xb4d', u'\u02cbd', u'^d', u'\xa8d', u'\u02c9d', u'_d', u'\u02d8d', u'\u0311d', u'\u02c7d', u'~d', u'\u02d9d', u'.d', u'\xb0d', u'\u02ddd', u'\u02f5d', u'\xb8d', u'\u02dbd', u',d', u'-d', u'/d', u'\u0131', u'\u0130', u'+', u'<', u'>', u'=', u'\xb1', u'\xd7', u'\xf7', u'\u2264', u'\u2265', u'\u2260', u'\u2243', u'\xb0', u'%', u'\u2030', u'\u221a', u'\u221e', u'\xbc', u'\xbd', u'\xbe', u'@', u'#', u'/', u'\\', u'|', u'.', u',', u';', u':', u'!', u'?', u'\xa1', u'\xbf', u'\u2026', u'-', u'\u2011', u'\u2014', u'\u2013', u'_', u'*', u'\u2020', u'\u2021', u'\xa7', u'(', u')', u'[', u']', u'{', u'}', u'\u201c', u'\u201d', u'\u2018', u'\u2019', u'\xab', u'\xbb', u'\u201a', u'\u201e', u'\u2039', u'\u203a', u'$', u'\xa3', u'\xa2', u'\xa4',

({u'!': 0.00067807639447807152,
  u'#': 4.0054151852749071e-05,
  u'$': 7.9827206808416868e-06,
  u'%': 0.00022123032787840644,
  u'&': 9.1661238136329993e-05,
  u'(': 0.0015574670491563922,
  u')': 0.0017116216386650835,
  u'*': 0.00044893881521192299,
  u'+': 6.6616322439612852e-05,
  u',': 0.0095089613028575305,
  u',d': 1.3126402722990883e-06,
  u'-': 0.0026215497527427062,
  u'-d': 1.2553494367308275e-09,
  u'.': 0.0082053706779437897,
  u'.d': 5.7085591608470613e-07,
  u'/': 0.00056749668669811213,
  u'/d': 2.1784707405602981e-10,
  u':': 0.0013507669693385231,
  u';': 0.00049261360286380199,
  u'<': 5.4913232356387941e-06,
  u'=': 5.1660220541394774e-05,
  u'>': 2.7737496765873932e-05,
  u'?': 0.0004625353838850703,
  u'@': 3.7824744825290083e-05,
  u'[': 9.1058138889767344e-05,
  u'\\': 7.1959579705030664e-06,
  u']': 0.00012457485390482705,
  u'^d': 0.0020190730357488962,
  u'_': 0.00030273561973937385,
  u'_d': 4.2801537910056179e-08,
  u'a': 0.060795412967058086,
  u'b': 0.0

In [6]:
bigram_file = "input/resources/Formals_all_Avg_freqs_2.tsv"
with codecs.open(bigram_file, 'r', encoding="utf-8") as bigram_file:
    all_lines = bigram_file.readlines()
    lines = [l.rstrip() for l in all_lines]
    #create dict
    p_bigrams = {}
    for i in range(0,len(lines)):
        l = lines[i]
        parts = l.split(" ")
        if len(parts) < 3:
            for j in range(i-10,i+1):
                print lines[j]
        p_bigrams[(parts[0], parts[1])] = float(parts[2])

In [12]:
u'space   5.955334137574368E-9'.split(" ")



SyntaxError: EOL while scanning string literal (<ipython-input-12-9dacc1a2887e>, line 1)

In [18]:
for for (c1,c2) in p_bigrams.keys():
    if c1 == u"r":
        print(c1 +','+ c2)

r,?
r,Ð
r,Ρ
r,¤
r,Η
r,Χ
r,Υ
r,σ
r,Þ
r,ǝ
r,z
r,.d
r,Δ
r,√
r,À
r,τ
r,ω
r,9
r,κ
r,r
r,\
r,†
r,7
r,v
r,Ξ
r,ʒ
r,˵d
r,æ
r,c
r,_
r,ρ
r,£
r,̑d
r,f
r,Β
r,]
r,ù
r,Œ
r,%
r,|
r,Λ
r,°
r,δ
r,ˇd
r,à
r,Φ
r,ι
r,p
r,›
r,Μ
r,0
r,‹
r,Ν
r,.
r,ψ
r,–
r,ξ
r,ſ
r,!
r,Π
r,Ʒ
r,1
r,»
r,ο
r,e
r,&
r,—
r,d
r,˝d
r,˙d
r,Ω
r,Σ
r,¾
r,$
r,β
r,‑
r,Ζ
r,„
r,π
r,/
r,l
r,{
r,<
r,Τ
r,ˉd
r,q
r,“
r,φ
r,η
r,ß
r,’
r,ζ
r,þ
r,§
r,Æ
r,}
r,¥
r,-d
r,˘d
r,b
r,Ŋ
r,…
r,=
r,;
r,é
r,Θ
r,×
r,_d
r,ŋ
r,‰
r,@
r,ç
r,¼
r,Ε
r,‡
r,space
r,ı
r,α
r,€
r,Ψ
r,ε
r,θ
r,Ǝ
r,j
r,¸d
r,ẞ
r,±
r,:
r,λ
r,h
r,)
r,¡
r,u
r,a
r,∞
r,^d
r,¨d
r,~d
r,İ
r,o
r,4
r,χ
r,«
r,2
r,θ2
r,÷
r,ª
r,ĳ
r,₩
r,®
r,Ĳ
r,Γ
r,¿
r,É
r,i
r,*
r,≃
r,m
r,,d
r,°d
r,½
r,k
r,,
r,≥
r,œ
r,3
r,è
r,g
r,(
r,¢
r,υ
r,≠
r,ˋd
r,/d
r,˛d
r,Ι
r,ð
r,x
r,·
r,>
r,≤
r,n
r,Κ
r,Ο
r,[
r,”
r,º
r,Α
r,-
r,t
r,´d
r,5
r,y
r,Ç
r,μ
r,#
r,©
r,‚
r,s
r,+
r,γ
r,È
r,w
r,8
r,ν
r,‘
r,6


In [7]:
p_bigrams[("M", "M")] += 2

KeyError: ('M', 'M')